<a href="https://colab.research.google.com/github/varuncornelio/Deep_Learning_Assignment/blob/main/deep_learning_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install torch==1.10.1+cu102 torchvision==0.11.2+cu102 torchaudio===0.10.1+cu102 -f https://download.pytorch.org/whl/cu102/torch_stable.html

Looking in links: https://download.pytorch.org/whl/cu102/torch_stable.html
     |██████████████████████████████▎ | 834.1 MB 1.2 MB/s eta 0:00:40tcmalloc: large alloc 1147494400 bytes == 0x55dcbf1fc000 @  0x7f30a8852615 0x55dc85bef4cc 0x55dc85ccf47a 0x55dc85bf22ed 0x55dc85ce3e1d 0x55dc85c65e99 0x55dc85c609ee 0x55dc85bf3bda 0x55dc85c65d00 0x55dc85c609ee 0x55dc85bf3bda 0x55dc85c62737 0x55dc85ce4c66 0x55dc85c61daf 0x55dc85ce4c66 0x55dc85c61daf 0x55dc85ce4c66 0x55dc85c61daf 0x55dc85bf4039 0x55dc85c37409 0x55dc85bf2c52 0x55dc85c65c25 0x55dc85c609ee 0x55dc85bf3bda 0x55dc85c62737 0x55dc85c609ee 0x55dc85bf3bda 0x55dc85c61915 0x55dc85bf3afa 0x55dc85c61c0d 0x55dc85c609ee
     |████████████████████████████████| 881.9 MB 15 kB/s 
     |████████████████████████████████| 23.2 MB 1.3 MB/s 
     |████████████████████████████████| 2.9 MB 11.1 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1

In [ ]:
!git clone https://github.com/12wang3/mllp.git

Cloning into 'mllp'...
remote: Enumerating objects: 55, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 55 (delta 7), reused 47 (delta 3), pack-reused 0
Unpacking objects: 100% (55/55), done.


In [ ]:
cd mllp

/content/mllp


In [ ]:
import torch
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

from mllp.models import MLLP
from mllp.utils import DBEncoder

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
data = load_breast_cancer()

X_df = pd.DataFrame(data['data'], columns=data['feature_names'])
y_df = pd.DataFrame(data['target'], columns=['class'])
f_df = pd.DataFrame(zip(data['feature_names'], ['continuous'] * len(data.feature_names)))
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, train_size=0.8)

In [ ]:
db_enc = DBEncoder(f_df, discrete=True)
db_enc.fit(X_train, y_train)
X_train, y_train = db_enc.transform(X_train, y_train)
X_test, y_test = db_enc.transform(X_test, y_test)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/content/mllp/mllp/discretizer.py:28: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  discrete_col = pd.DataFrame(pd.np.digitize(continuous_data[k], bins=self.boundaries[k], right=True),
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  

In [ ]:
db_enc.me_discretizer.boundaries

defaultdict(list,
            {'area error': [31.24, 53.65],
             'compactness error': [0.01084, 0.02213],
             'concave points error': [0.009199],
             'concavity error': [0.01245, 0.02095],
             'fractal dimension error': [0.00233],
             'mean area': [471.3, 693.7],
             'mean compactness': [0.1021],
             'mean concave points': [0.04908, 0.07857],
             'mean concavity': [0.07135, 0.103],
             'mean fractal dimension': [0.05664],
             'mean perimeter': [85.24, 98.73, 106.6],
             'mean radius': [12.32, 15.04, 17.85],
             'mean smoothness': [0.08946],
             'mean symmetry': [0.1714],
             'mean texture': [18.45],
             'perimeter error': [1.714, 2.759, 5.118],
             'radius error': [0.1924, 0.4202, 0.5462],
             'smoothness error': [],
             'symmetry error': [0.04243],
             'texture error': [],
             'worst area': [694.4, 873.2, 12

In [ ]:
net_structure = [X_train.shape[-1], 32, y_train.shape[-1]]
# below is a more complex MLLP structure that can be used for a complex data set
# net_structure = [X_train.shape[-1], 128, 128, 64, y_train.shape[-1]]
net = MLLP(net_structure,
           device=device,
           random_binarization_rate=0.0,
           use_not=False)
net.to(device)

MLLP(
  (conj0): ConjunctionLayer(
    (randomly_binarize_layer): RandomBinarizationLayer()
  )
  (disj0): DisjunctionLayer(
    (randomly_binarize_layer): RandomBinarizationLayer()
  )
)

In [ ]:
training_log = net.train(
    X_train,
    y_train,
    lr=0.005,
    batch_size=16,
    epoch=100,
    lr_decay_rate=0.75,
    lr_decay_epoch=100,
    weight_decay=1e-7)

[INFO] - LR is set to 0.005
[INFO] - epoch: 0, loss: 6.861445292830467
[INFO] - ------------------------------------------------------------
[INFO] - On Training Set:
	Accuracy of MLLP Model: 0.2857142857142857
	Accuracy of CRS  Model: 0.7142857142857143
[INFO] - On Training Set:
	F1 Score of MLLP Model: 0.22222222222222224
	F1 Score of CRS  Model: 0.41666666666666663
[INFO] - ------------------------------------------------------------
[INFO] - epoch: 1, loss: 3.084330443292856
[INFO] - epoch: 2, loss: 2.0992393530905247
[INFO] - epoch: 3, loss: 1.7411252968013287
[INFO] - epoch: 4, loss: 1.5871057864278555
[INFO] - epoch: 5, loss: 1.4145475793629885
[INFO] - ------------------------------------------------------------
[INFO] - On Training Set:
	Accuracy of MLLP Model: 1.0
	Accuracy of CRS  Model: 0.2857142857142857
[INFO] - On Training Set:
	F1 Score of MLLP Model: 1.0
	F1 Score of CRS  Model: 0.22222222222222224
[INFO] - ------------------------------------------------------------
[

In [ ]:
acc, acc_b, f1, f1_b = net.test(X_test, y_test, need_transform=True)

print('Accuracy of MLLP Model: {}'
      '\nAccuracy of CRS  Model: {}'
      '\nF1 Score of MLLP Model: {}'
      '\nF1 Score of CRS  Model: {}'.format(acc, acc_b, f1, f1_b))

Accuracy of MLLP Model: 0.956140350877193
Accuracy of CRS  Model: 0.9385964912280702
F1 Score of MLLP Model: 0.9539245008487591
F1 Score of CRS  Model: 0.9359910162829872


In [ ]:
net.concept_rule_set_print(X_fname=db_enc.X_fname, y_fname=db_enc.y_fname, eliminate_redundancy=True)

------------------------------------------------------------------------------------------
 class_0:
	       r1,3:	 [' mean texture_>18.45', ' mean perimeter_(85.24, 98.73]', ' area error_(31.24, 53.65]', ' worst texture_>25.82', ' worst concavity_(0.206, 0.3779]']
	       r1,5:	 [' mean texture_>18.45', ' mean concavity_>0.103', ' worst texture_>25.82', ' worst smoothness_>0.136']
	       r1,8:	 [' mean radius_(12.32, 15.04]', ' mean smoothness_>0.08946', ' worst perimeter_(101.6, 117.2]', ' worst smoothness_>0.136', ' worst fractal dimension_>0.09585']
	      r1,12:	 [' mean radius_(12.32, 15.04]', ' mean smoothness_>0.08946', ' mean concavity_>0.103', ' mean concave points_>0.07857', ' perimeter error_(1.714, 2.759]', ' worst smoothness_>0.136']
	      r1,17:	 [' mean concavity_>0.103', ' mean concave points_>0.07857', ' worst perimeter_>117.2', ' worst smoothness_>0.136']
	      r1,22:	 [' mean area_>693.7', ' worst perimeter_>117.2']
	      r1,24:	 [' mean texture_>18.45', ' mean 

In [ ]:
!python experiments.py -d tic-tac-toe &

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  "X does not have valid feature names, but"


In [ ]:
!tail -f log_folder/tic-tac-toe_k5_ki0_useValidationSetFalse_e401_bs64_lr0.01_lrdr0.75_lrde100_wd0.0_p0.0_useNOTFalse_L64/log.txt

	F1 Score of CRS  Model: 1.0
[INFO] - ------------------------------------------------------------
[INFO] - ============================================================
[INFO] - Test:
	Accuracy of MLLP Model: 0.9895833333333334
	Accuracy of CRS  Model: 0.9947916666666666
[INFO] - Test:
	F1 Score of MLLP Model: 0.989158667419537
	F1 Score of CRS  Model: 0.9946126434523976
[INFO] - ============================================================
^C


In [ ]:
# adult data set
# You'd better run it on GPU for training large network on CPU may cost lots of time.
!python experiments.py -d adult -e 800 -bs 64 -lr 0.005 -p 0.9 --use_not --use_validation_set -s 256_256_64 &

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/content/mllp/mllp/discretizer.py:28: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  discrete_col = pd.DataFrame(pd.np.digitize(continuous_data[k], bins=self.boundaries[k], right=True),
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  

In [ ]:
from google.colab import files
files.download("/content/mllp/log_folder")